# MVD 6. cvičení

## 1. část - PageRank

Data k dnešnímu cvičení použijte z tohoto [Github repozitáře](https://github.com/chonyy/PageRank-HITS-SimRank/tree/master/dataset). Konkrétně nás budou zajímat soubory *graph_1.txt* až *graph_6.txt*. K daným datasetům je v repozitáři implementace PageRank algoritmu, každopádně se touto implementací nijak neinspirujte. 

Cílem je naprogramovat PageRank vektorizovaně podle přednášky, povoleno je pouze použití knihovny numpy. Parametr $\alpha$ nastavte na hodnotu 0.2 a počet iterací bude 100. U prvních grafů uvidíte, že PageRank konverguje mnohem dříve a u těch složitějších nemusí stačit ani 100 iterací.

<br>
$
p^{(0)} = (\frac{1}{N}, ..., \frac{1}{N})^T \\
A = ((1-\alpha)M + \frac{\alpha}{N}E) \\
Opakujte: \\
\hspace{1cm}p^{(i+1)} = A^Tp^{(i)}
$

Pozor: Stránka, která nemá výstupní linky, musí mít nastavený parametr $\alpha$ na 1.

In [1]:
import numpy as np

In [7]:
def get_A_and_ordered_uniq_nodes(filename):
    links = {}
    default_alpha = 0.2
    uniq_nodes = set()

    with open(filename) as file:
        for line in file.readlines():
            link_from, link_to = line.strip().split(',')
            uniq_nodes.add(link_from)
            uniq_nodes.add(link_to)

            if link_from in links:
                links[link_from].add(link_to)
            else:
                links[link_from] = {link_to}

    ordered_uniq_nodes = sorted(list(uniq_nodes))
    from2idx = {} # reverse index
    for i, link_from in enumerate(ordered_uniq_nodes):
        from2idx[link_from] = i

    # create M matrix
    AA = np.zeros((len(ordered_uniq_nodes), len(ordered_uniq_nodes)))
    M = np.zeros((len(ordered_uniq_nodes), len(ordered_uniq_nodes))) # M
    for link_from, links_to in links.items():
        link_from_idx = from2idx[link_from]

        for link_to in list(links_to):
            link_to_idx = from2idx[link_to]
            AA[link_from_idx, link_to_idx] = 1
            M[link_from_idx, link_to_idx] = 1 / len(links_to)

    alpha = np.ones((len(ordered_uniq_nodes), 1))
    alpha[np.sum(M, axis=1) != 0] = default_alpha

    print('M', (1 - alpha) * M)

    A = (1 - alpha) * M + (alpha / len(ordered_uniq_nodes)) * np.ones(len(ordered_uniq_nodes))

    return A, AA, ordered_uniq_nodes

In [8]:
file2rank = {}
for file_number in range(1, 2):
    filename = f'graph_{file_number}.txt'
    A, _, ordered_uniq_nodes = get_A_and_ordered_uniq_nodes(filename)

    page_rank = np.ones((len(ordered_uniq_nodes), 1)) / len(ordered_uniq_nodes)
    for _ in range(100):
        page_rank = np.dot(A.T, page_rank)

    file2rank[filename] = page_rank

file2rank

M [[0.  0.8 0.  0.  0.  0. ]
 [0.  0.  0.8 0.  0.  0. ]
 [0.  0.  0.  0.8 0.  0. ]
 [0.  0.  0.  0.  0.8 0. ]
 [0.  0.  0.  0.  0.  0.8]
 [0.  0.  0.  0.  0.  0. ]]


{'graph_1.txt': array([[0.0656044 ],
        [0.11808792],
        [0.16007474],
        [0.19366419],
        [0.22053575],
        [0.242033  ]])}

### Předpokládaný výstup

#### graph_1.xt
```python
array([[0.0656044 ],
       [0.11808792],
       [0.16007474],
       [0.19366419],
       [0.22053575],
       [0.242033  ]])
```
       
#### graph_2.txt
```python
array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])
```


#### graph_3.txt
```python
array([[0.17857143],
       [0.32142857],
       [0.32142857],
       [0.17857143]])
```


#### graph_4.txt
```python
array([[0.27257372],
       [0.15666713],
       [0.13837881],
       [0.10924643],
       [0.18531604],
       [0.06563464],
       [0.07218322]])
```


## 2. část - HITS

Použijte stejná data jako u PageRank algoritmu a počet iterací bude opět 100.

Implementujte dle následujícího algoritmu:
<br>

$
a^{(0)} = (1, ..., 1)^T, h^{(0)} = (1, ..., 1)^T
\\
Opakujte:\\
    \hspace{1cm} h^{(i+1)} = Aa^{(i)}\\
    \hspace{1cm} h^{(i+1)} = \frac{h^{(i+1)}}{||h^{(i+1)}||_1}\\
    \hspace{1cm} a^{(i+1)} = A^Th^{(i)}\\
    \hspace{1cm} a^{(i+1)} = \frac{a^{(i+1)}}{||a^{(i+1)}||_1}\\   
$

In [100]:
file2hits = {}
for file_number in range(1, 7):
    filename = f'graph_{file_number}.txt'
    _, A, ordered_uniq_nodes = get_A_and_ordered_uniq_nodes(filename)
    a = np.ones((len(ordered_uniq_nodes), 1))
    hubs = np.ones(a.shape)

    for _ in range(100):
        tmp_old_hubs = hubs
        hubs = A @ a
        hubs /= np.linalg.norm(hubs, ord=1)
        a = A.T @ tmp_old_hubs
        a /= np.linalg.norm(a, ord=1)

    file2hits[filename] = {
        'Hub': hubs,
        'Authority': a,
    }

file2hits

{'graph_1.txt': {'Hub': array([[0.2],
         [0.2],
         [0.2],
         [0.2],
         [0.2],
         [0. ]]),
  'Authority': array([[0. ],
         [0.2],
         [0.2],
         [0.2],
         [0.2],
         [0.2]])},
 'graph_2.txt': {'Hub': array([[0.2],
         [0.2],
         [0.2],
         [0.2],
         [0.2]]),
  'Authority': array([[0.2],
         [0.2],
         [0.2],
         [0.2],
         [0.2]])},
 'graph_3.txt': {'Hub': array([[0.19098301],
         [0.30901699],
         [0.30901699],
         [0.19098301]]),
  'Authority': array([[0.19098301],
         [0.30901699],
         [0.30901699],
         [0.19098301]])},
 'graph_4.txt': {'Hub': array([[0.27545318],
         [0.04776231],
         [0.10868324],
         [0.19865956],
         [0.1837346 ],
         [0.11673471],
         [0.06897241]]),
  'Authority': array([[0.13948389],
         [0.17791203],
         [0.20082321],
         [0.14017775],
         [0.20142536],
         [0.05608926],
        

### Předpokládaný výstup

#### graph_1.xt
```python
Authority:[[0. ]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]
 [0. ]]
```
       
#### graph_2.txt
```python
Authority:[[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
Hub: [[0.2]
 [0.2]
 [0.2]
 [0.2]
 [0.2]]
```


#### graph_3.txt
```python
Authority:[[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
Hub: [[0.19098301]
 [0.30901699]
 [0.30901699]
 [0.19098301]]
```


#### graph_4.txt
```python
Authority:[[0.13948389]
 [0.17791203]
 [0.20082321]
 [0.14017775]
 [0.20142536]
 [0.05608926]
 [0.08408849]]
Hub: [[0.27545318]
 [0.04776231]
 [0.10868324]
 [0.19865956]
 [0.1837346 ]
 [0.11673471]
 [0.06897241]]
```


## Bonus - Invertovaný index pomocí MapReduce

Bonusovou úlohou je vytvoření invertovaného indexu stejně, jako je uvedeno na příkladu v přednášce. Implementace nebude v standardním MapReduce frameworku, ale použijete python funkce ```map()``` a ```reduce()```. Funkci map lze poté spustit paralelně s pomocí ```Pool``` objektu z knihovny ```multiprocessing```. 

Vstupními daty budou Medium články, které jsme používali v posledních pár cvičeních. Z těchto článků použijte pouze nadpisy (title). 

In [87]:
# Instalace spaCy z Jupyter Notebooku
import sys

import numpy as np
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

     |████████████████████████████████| 12.0 MB 9.9 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/en_core_web_sm
-->
/Users/admin/opt/anaconda3/envs/MVD_2021/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [88]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [89]:
import pandas as pd

df = pd.read_csv('../cv04/articles.csv')
df.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If you’ve ever found yourself looking up the s...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python – Towa...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch – Insight ...,Want to learn about applied Artificial Intelli...


In [90]:
import string

def lower(s: str) -> str:
    return s.lower()

def strip_punctuation(s: str) -> str:
    return s.translate(str.maketrans('', '', string.punctuation))

def lemmanize(s: str) -> str:
    return ' '.join([token.lemma_ for token in lemmatizer(s)])

def preprocess(df: pd.DataFrame) -> None:
    for column in ['author', 'title', 'text', 'claps']:
        df[column] = df[column].apply(lower).apply(strip_punctuation).apply(lemmanize)

In [91]:
preprocess(df)
df.head()

,author,claps,reading_time,link,title,text
0,justin lee,83k,11,https://medium.com/swlh/chatbots-were-the-next...,chatbot be the next big thing what happen – th...,oh how the headline blare \n chatbot be the ne...
1,conor dewey,14k,7,https://towardsdatascience.com/python-for-data...,python for data science 8 concept -PRON- may h...,if -PRON- have ever find -PRON- look up the sa...
2,william koehrsen,28k,11,https://towardsdatascience.com/automated-featu...,automate feature engineering in python – towar...,machine learning be increasingly move from han...
3,gant laborde,13k,7,https://medium.freecodecamp.org/machine-learni...,machine learn how to go from zero to hero – fr...,if -PRON- understanding of ai and machine lear...
4,emmanuel ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,reinforcement learn from scratch – insight datum,want to learn about applied artificial intelli...


In [92]:
def make_inv_index(data):
    word_to_idx = {}
    idx, title = data[0], data[1]
    for word in title.split():
        if not word in word_to_idx:
            word_to_idx[word] = {idx}
        else:
            word_to_idx[word].add(idx)

    return word_to_idx

In [93]:
def reduce_inv_index(a, b):
    reduced = a
    for word, indices in b.items():
        if word in reduced:
            reduced[word] = reduced[word].union(indices)
        else:
            reduced[word] = indices

    return reduced

In [94]:
from functools import reduce

In [95]:
indices = np.array(df.index.to_list()).T
indices

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [96]:
titles = np.array(df.title.to_list())
titles

array(['chatbot be the next big thing what happen – the startup – medium',
       'python for data science 8 concept -PRON- may have forget',
       'automate feature engineering in python – towards data science',
       'machine learn how to go from zero to hero – freecodecamp',
       'reinforcement learn from scratch – insight datum',
       'intuitively understanding convolution for deep learning',
       'an intro to machine learning for designer – ux collective',
       'the big list of dsml interview resource – towards data science',
       'must know information theory concept in deep learning ai',
       'what i learn from interview at multiple ai company and startup',
       'from ballerina to ai researcher part i – buzzrobot',
       '3 way to apply latent semantic analysis on largecorpus text on macos terminal jupyterlab and',
       'deep learning be go to teach -PRON- all the lesson of -PRON- life job be for machine',
       'machine learning be fun – adam geitgey – mediu

In [97]:
data = np.column_stack((indices, titles))
data

array([['0',
        'chatbot be the next big thing what happen – the startup – medium'],
       ['1', 'python for data science 8 concept -PRON- may have forget'],
       ['2',
        'automate feature engineering in python – towards data science'],
       ['3', 'machine learn how to go from zero to hero – freecodecamp'],
       ['4', 'reinforcement learn from scratch – insight datum'],
       ['5', 'intuitively understanding convolution for deep learning'],
       ['6', 'an intro to machine learning for designer – ux collective'],
       ['7',
        'the big list of dsml interview resource – towards data science'],
       ['8', 'must know information theory concept in deep learning ai'],
       ['9',
        'what i learn from interview at multiple ai company and startup'],
       ['10', 'from ballerina to ai researcher part i – buzzrobot'],
       ['11',
        '3 way to apply latent semantic analysis on largecorpus text on macos terminal jupyterlab and'],
       ['12',
        '

In [98]:
mapped_inverted_index = list(map(make_inv_index, data))
inverted_index = reduce(reduce_inv_index, mapped_inverted_index)

In [99]:
inverted_index

{'chatbot': {'0', '192', '223', '257', '267', '273', '86', '96'},
 'be': {'0',
  '108',
  '113',
  '12',
  '121',
  '123',
  '128',
  '13',
  '136',
  '138',
  '139',
  '14',
  '140',
  '15',
  '153',
  '169',
  '170',
  '18',
  '216',
  '218',
  '224',
  '227',
  '231',
  '233',
  '237',
  '243',
  '246',
  '258',
  '261',
  '264',
  '267',
  '273',
  '321',
  '330',
  '335',
  '36',
  '56',
  '57',
  '58',
  '60',
  '61',
  '83',
  '86',
  '87',
  '98'},
 'the': {'0',
  '100',
  '102',
  '104',
  '105',
  '112',
  '117',
  '12',
  '121',
  '124',
  '127',
  '128',
  '130',
  '139',
  '140',
  '141',
  '142',
  '143',
  '147',
  '149',
  '154',
  '155',
  '163',
  '165',
  '17',
  '175',
  '177',
  '180',
  '182',
  '183',
  '189',
  '19',
  '20',
  '201',
  '204',
  '207',
  '208',
  '21',
  '211',
  '217',
  '218',
  '219',
  '224',
  '226',
  '229',
  '230',
  '232',
  '24',
  '241',
  '242',
  '244',
  '246',
  '248',
  '25',
  '254',
  '255',
  '256',
  '258',
  '260',
  '262',
 

### Předpokládaný výstup

```python
print(inverted_index['be'])
print(titles[12]) # zobrazení nadpisu pro kontrolu
```
```
[('D0', 1), ('D12', 2), ('D13', 1), ('D14', 1), ('D15', 1), ('D18', 1), ('D36', 1), ('D56', 1), ('D57', 1), ('D58', 1), ('D60', 1), ('D61', 1), ('D83', 1), ('D86', 1), ('D87', 1), ('D98', 1), ('D108', 1), ('D113', 1), ('D121', 1), ('D123', 2), ('D128', 1), ('D136', 1), ('D138', 1), ('D139', 1), ('D140', 1), ('D153', 1), ('D169', 1), ('D170', 1), ('D216', 1), ('D218', 1), ('D224', 1), ('D227', 1), ('D231', 1), ('D233', 1), ('D237', 1), ('D246', 1), ('D258', 1), ('D261', 1), ('D264', 1), ('D267', 1), ('D273', 1), ('D321', 1), ('D330', 1), ('D335', 1)]
```
deep learning be go to teach we all the lesson of our life job be for machine

Výstup bude identický za předpokladu použití spacy lemmatizéru. Zároveň výstup nemusí obsahovat stejný formát indexu, postačí *(index, count)*.